# Working with Data Sets


In [7]:
import pandas as pd
import numpy as np
import datetime

In [31]:
Films=pd.read_csv('datasets\movies.csv', sep=',',decimal=".")

In [10]:
Films.head()

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description
0,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...
1,2,The Dark Knight,2008,9.0,84.0,"Action, Crime, Drama",1980200,Christopher Nolan,152,534.86,When the menace known as the Joker emerges fro...
2,3,Inception,2010,8.8,74.0,"Action, Adventure, Sci-Fi",1760209,Christopher Nolan,148,292.58,A thief who steals corporate secrets through t...
3,4,Fight Club,1999,8.8,66.0,Drama,1609459,David Fincher,139,37.03,An insomniac office worker and a devil-may-car...
4,5,Pulp Fiction,1994,8.9,94.0,"Crime, Drama",1570194,Quentin Tarantino,154,107.93,"The lives of two mob hitmen, a boxer, a gangst..."


In [32]:
Oscar=pd.read_csv('datasets/the_oscar_award.csv')

In [14]:
Oscar.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


### Первый данные с фильмами не достаточны для нас, так как там нет записи об актерах, которые нужны для оценки популярности фильма


In [33]:
fulltable = pd.merge(left=Films, right=Oscar[['year_film','year_ceremony','category','name', 'film','winner']], left_on= ['Title','Year'], right_on =['film','year_film'], how='left')
fulltable.head()


,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description,year_film,year_ceremony,category,name,film,winner
0,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,ACTOR IN A LEADING ROLE,Morgan Freeman,The Shawshank Redemption,False
1,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,CINEMATOGRAPHY,Roger Deakins,The Shawshank Redemption,False
2,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,FILM EDITING,Richard Francis-Bruce,The Shawshank Redemption,False
3,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,MUSIC (Original Score),Thomas Newman,The Shawshank Redemption,False
4,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,BEST PICTURE,"Niki Marvin, Producer",The Shawshank Redemption,False


In [34]:
d = {False : 2, True : 1, np.nan : 0}
fulltable['winner'] = fulltable['winner'].map(d)
fulltable.head()

,Rank,Title,Year,Score,Metascore,Genre,Vote,Director,Runtime,Revenue,Description,year_film,year_ceremony,category,name,film,winner
0,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,ACTOR IN A LEADING ROLE,Morgan Freeman,The Shawshank Redemption,2
1,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,CINEMATOGRAPHY,Roger Deakins,The Shawshank Redemption,2
2,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,FILM EDITING,Richard Francis-Bruce,The Shawshank Redemption,2
3,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,MUSIC (Original Score),Thomas Newman,The Shawshank Redemption,2
4,1,The Shawshank Redemption,1994,9.3,80.0,Drama,2011509,Frank Darabont,142,28.34,Two imprisoned men bond over a number of years...,1994.0,1995.0,BEST PICTURE,"Niki Marvin, Producer",The Shawshank Redemption,2


In [39]:
from sklearn.model_selection import train_test_split
fulltable=fulltable.dropna()
train, test = train_test_split(fulltable, test_size=0.2)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 676 entries, 11145 to 612
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           676 non-null    int64  
 1   Title          676 non-null    object 
 2   Year           676 non-null    int64  
 3   Score          676 non-null    float64
 4   Metascore      676 non-null    float64
 5   Genre          676 non-null    object 
 6   Vote           676 non-null    int64  
 7   Director       676 non-null    object 
 8   Runtime        676 non-null    int64  
 9   Revenue        676 non-null    float64
 10  Description    676 non-null    object 
 11  year_film      676 non-null    float64
 12  year_ceremony  676 non-null    float64
 13  category       676 non-null    object 
 14  name           676 non-null    object 
 15  film           676 non-null    object 
 16  winner         676 non-null    int64  
dtypes: float64(5), int64(5), object(7)
memory usage: 9

In [26]:
from sklearn.linear_model import LogisticRegression
X_train = train[['Score', 'Revenue', 'Vote','Runtime']]
Y_train = train.winner
X_test = test[['Score', 'Revenue', 'Vote','Runtime']]
Y_test = test.winner
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
from sklearn.metrics import recall_score, precision_score
print('Score: ' + str(logreg.score(X_test, Y_test)))
print('Precision: ' + str(precision_score(Y_pred, Y_test)))
print('Recall: ' + str(recall_score(Y_pred, Y_test)))

Score: 0.7603550295857988
Precision: 0.006535947712418301
Recall: 0.09090909090909091


In [36]:
from sklearn.linear_model import LogisticRegression
X_train = train[['Score', 'Revenue']]
Y_train = train.winner
X_test = test[['Score', 'Revenue']]
Y_test = test.winner
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
from sklearn.metrics import recall_score, precision_score
print('Score: ' + str(logreg.score(X_test, Y_test)))
print('Precision: ' + str(precision_score(Y_pred, Y_test)))
print('Recall: ' + str(recall_score(Y_pred, Y_test)))

Score: 0.7470414201183432
Precision: 0.024242424242424242
Recall: 0.2857142857142857


In [40]:
from sklearn.linear_model import LogisticRegression
X_train = train[['Score','Vote']]
Y_train = train.winner
X_test = test[['Score','Vote']]
Y_test = test.winner
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
from sklearn.metrics import recall_score, precision_score
print('Score: ' + str(logreg.score(X_test, Y_test)))
print('Precision: ' + str(precision_score(Y_pred, Y_test)))
print('Recall: ' + str(recall_score(Y_pred, Y_test)))

Score: 0.7662721893491125
Precision: 0.0
Recall: 0.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
from sklearn.linear_model import LogisticRegression
X_train = train[['Score','Vote']]
Y_train = train.winner
X_test = test[['Score','Vote']]
Y_test = test.winner
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
from sklearn.metrics import recall_score, precision_score
print('Score: ' + str(logreg.score(X_test, Y_test)))
print('Precision: ' + str(precision_score(Y_pred, Y_test)))
print('Recall: ' + str(recall_score(Y_pred, Y_test)))

Score: 0.7736686390532544
Precision: 0.0
Recall: 0.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
from sklearn.neighbors import KNeighborsClassifier 

# Create arrays for the features and the response variable
y = train['winner'].values
X = train[['Score','Vote','Runtime','Revenue']].values

# Create a k-NN classifier with 6 neighbors
knn = KNeighborsClassifier(n_neighbors=6)

# Fit the classifier to the data
knn.fit(X, y)
knn.predict(test['winner'].values).head()

ValueError: Expected 2D array, got 1D array instead:
array=[2 2 2 1 2 1 2 1 2 2 1 2 1 2 2 1 2 2 2 2 2 2 2 2 2 1 1 2 1 1 2 2 2 1 1 2 2
 2 2 1 2 2 2 2 2 2 1 1 1 1 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2
 1 2 2 2 1 1 2 2 2 1 1 2 2 2 1 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2
 2 1 2 2 1 1 2 1 2 2 1 2 2 2 1 2 1 1 2 2 2 1 1 2 2 2 2 2 2 2 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 1 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 2 2 2 2 1 2 2 2
 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 1 2 2 2 2 2
 2 1 1 2 2 2 2 2 2 1 2 2 2 1 2 2 2 1 1 2 2 2 2 2 2 2 2 1 2 1 2 1 2 2 2 1 2
 2 2 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1
 2 1 2 2 2 1 2 2 2 2 2 2 1 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2
 2 1 1 2 2 1 2 1 2 2 2 2 2 1 2 2 2 2 2 2 2 1 2 2 2 2 2 1 1 2 2 2 1 2 2 2 2
 2 1 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 1 2 2 1 2 2 2 2 2 1 2 2 2 2 1 2 2
 2 2 1 2 2 2 1 2 2 2 2 2 1 2 1 2 1 2 2 1 1 2 1 1 2 2 2 1 2 1 1 2 2 1 1 1 1
 2 2 2 2 1 2 1 2 1 2 2 1 2 2 1 2 2 1 2 1 